<a href="https://colab.research.google.com/github/joycerlz/distilbert-text-classification/blob/main/clean_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

In [25]:
df1 = pd.read_csv('big_five_train_set.csv')
df2 = pd.read_csv('big_five_eval_set.csv')
df3 = pd.read_csv('big_five_val_set.csv')

In [26]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df.sample(5)

,text,agreeableness,openness,conscientiousness,extraversion,neuroticism
17926,out of curiosity do you live near a body of water,46.0,27.0,31.0,10.0,84.0
13037,i accept help when i ask for it if i need it b...,15.0,50.0,15.0,15.0,50.0
1919,man those cobwebs are just the worst,9.0,61.0,13.0,4.0,72.0
9193,typemention are odd extroverts in that while w...,0.0,38.0,43.0,77.0,12.0
3339,exactly i do not care in the context of the joke,8.0,45.0,6.0,59.0,92.0


In [27]:
df.shape

(20984, 6)

## To multi-label

In [ ]:
def convert_multilabels(row):
  labels = [0] * 5
  if int(row['agreeableness']) >= 75:
    labels[0] = 1  # Agreeableness
  if int(row['extraversion']) >= 75:
    labels[1] = 1  # Extraversion
  if int(row['openness']) >= 75:
    labels[2] = 1  # Openness
  if int(row['conscientiousness']) >= 75:
    labels[3] = 1  # Conscientiousness
  if int(row['neuroticism']) >= 75:
    labels[4] = 1  # Neuroticism
  return labels

In [ ]:
df['labels'] = df.apply(convert_multilabels, axis=1)

# remove rows which all labels are 0
df = df[df['labels'].apply(lambda x: not all(v == 0 for v in x))]

df.drop(columns=['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism'], inplace=True)
df.head()

<ipython-input-20-68d3b8e774ba>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['agreeableness', 'openness', 'conscientiousness', 'extraversion', 'neuroticism'], inplace=True)


,text,labels
1,haha thank you i love this shit just all caps ...,"[1, 0, 0, 0, 0]"
2,twas either that or skin my chaffeurs,"[0, 0, 1, 0, 0]"
3,oooh i see what you were saying kind of doubt ...,"[0, 0, 1, 0, 1]"
4,not necessarily the case at all do you think h...,"[0, 0, 1, 0, 0]"
5,i really enjoyed reading your answers clear ty...,"[0, 0, 0, 0, 1]"


In [ ]:
df.shape

(20984, 2)

In [ ]:
df.to_csv('kaggle_cleaned.csv', index=False)

In [ ]:
df.to_csv('kaggle_cleaned_full.csv', index=False)

## To binary for each of the 5 types

In [30]:
df.head()

,text,agreeableness,openness,conscientiousness,extraversion,neuroticism
0,member member all the feels,9.0,61.0,13.0,4.0,72.0
1,haha thank you i love this shit just all caps ...,93.0,30.0,40.0,12.0,67.0
2,twas either that or skin my chaffeurs,38.0,86.0,10.0,3.0,69.0
3,oooh i see what you were saying kind of doubt ...,50.0,85.0,15.0,50.0,85.0
4,not necessarily the case at all do you think h...,38.0,86.0,10.0,3.0,69.0


In [31]:
df.rename(columns={'agreeableness': 'isAgree',
                   'openness': 'isOpen',
                   'conscientiousness': 'isCons',
                   'extraversion': 'isExtra',
                   'neuroticism': 'isNeuro'}, inplace=True)

In [37]:
df.head()

,text,isAgree,isOpen,isCons,isExtra,isNeuro
0,member member all the feels,9.0,61.0,13.0,4.0,72.0
1,haha thank you i love this shit just all caps ...,93.0,30.0,40.0,12.0,67.0
2,twas either that or skin my chaffeurs,38.0,86.0,10.0,3.0,69.0
3,oooh i see what you were saying kind of doubt ...,50.0,85.0,15.0,50.0,85.0
4,not necessarily the case at all do you think h...,38.0,86.0,10.0,3.0,69.0


In [80]:
# create copies for 5 personalities
df_agr = df.copy()
df_opn = df.copy()
df_con = df.copy()
df_ext = df.copy()
df_neu = df.copy()


df_agr.drop(columns=['isOpen', 'isCons', 'isExtra', 'isNeuro'], inplace=True)
df_opn.drop(columns=['isAgree', 'isCons', 'isExtra', 'isNeuro'], inplace=True)
df_con.drop(columns=['isOpen', 'isAgree', 'isExtra', 'isNeuro'], inplace=True)
df_ext.drop(columns=['isOpen', 'isCons', 'isAgree', 'isNeuro'], inplace=True)
df_neu.drop(columns=['isOpen', 'isCons', 'isExtra', 'isAgree'], inplace=True)

In [74]:
df_agr.head()

,text,isAgree
0,member member all the feels,9.0
1,haha thank you i love this shit just all caps ...,93.0
2,twas either that or skin my chaffeurs,38.0
3,oooh i see what you were saying kind of doubt ...,50.0
4,not necessarily the case at all do you think h...,38.0


In [81]:
def convert_label(val):
  if val >= 75:
    return 1
  else:
    return 0

In [82]:
df_agr['isAgree'] = df_agr['isAgree'].apply(convert_label)
df_opn['isOpen'] = df_opn['isOpen'].apply(convert_label)
df_con['isCons'] = df_con['isCons'].apply(convert_label)
df_ext['isExtra'] = df_ext['isExtra'].apply(convert_label)
df_neu['isNeuro'] = df_neu['isNeuro'].apply(convert_label)

In [87]:
df_neu.head()

,text,isNeuro
0,member member all the feels,0
1,haha thank you i love this shit just all caps ...,0
2,twas either that or skin my chaffeurs,0
3,oooh i see what you were saying kind of doubt ...,1
4,not necessarily the case at all do you think h...,0


In [92]:
df_agr.to_csv('kaggle_agr.csv', index=False)
df_opn.to_csv('kaggle_opn.csv', index=False)
df_con.to_csv('kaggle_con.csv', index=False)
df_ext.to_csv('kaggle_ext.csv', index=False)
df_neu.to_csv('kaggle_neu.csv', index=False)